In [1]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import joblib

from tqdm import tqdm_notebook, tqdm
from collections import deque

In [2]:
class AI_Feature():
    
    def __init__(self, state_size, action_space=2, model_name="TestM"): # action = feature 값을 올린다, 떨어트린다
        self.state_size = state_size
        self.action_space = action_space
        
        # 모델 파라미터
        self.memory = deque(maxlen=2000)
        self.inventory = []
        self.model_name = model_name
        
        self.gamma = 0.95 # 장기간 보상 대비 현재 보상을 최대화
        self.epsilon = 1.0 # 무작위 행동 or 모델을 사용할지
        self.epsilon_final = 0.01 # 
        self.epsilon_decay = 0.995
        
        self.model = self.model_builder() 
        
    def model_builder(self):
        model = tf.keras.models.Sequential()
        
        model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
        
        model.add(tf.keras.layers.Dense(units=64, activation='relu'))
        
        model.add(tf.keras.layers.Dense(units=128, activation='relu'))
        
        model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
        
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=1e-3))
        
        return model
    
    # 특정 상태에서 수행할 행동
    def trade(self, state):
        """
        1. 상태를 입력으로 가져와 무작위 숫자 생성
        2. 무작위수가 epsilon보다 작거나 같으면 무작위 행동 수행(처음에는 무조건 작음) => returen이 우리가 할 행동
        3. 무작위수가 epsilon보다 큰 경우 가능성이 높은 행동 반환
        """        
        # 무작위수가 epsilon보다 작거나 같으면 무작위 행동 수행
        # 무작위 행동 => 값을 올리거나 떨어트리거나
        if random.random() <= self.epsilon:
            return random.randrange(self.action_space)
        # 무작위수가 epsilon보다 큰 경우 모델을 사용할때 수행할 행동
        actions = self.model.predict(state) # 상태를 예측
        return np.argmax(actions[0]) # 가장 가능성이 높은 행동 반환
    
    
    def batch_trade(self, batch_szie):
        
        batch = []
        
        for i in range(len(self.memory) - batch_szie + 1, len(self.memory)):
            batch.append(self.memory[i])
        
        for state, action, reward, next_state, done in batch:
            reward = reward
            if not done: # agent가 종료상태가 아님을 확인, 
                reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0]) # 모델로 예측한 목표변수를 정의한 후의 할인된 총 보상
                
            target = self.model.predict(state) # 행동을 현재보상으로 수정(mse를 쓰는 이유)
            target[0][action] = reward # 
            
            self.model.fit(state, target, epochs=1, verbose=0) # 배치마다 모델을 학습
            
        if self.epsilon > self.epsilon_final: # 앱실론 파라미터 감소
            self.epsilon *= self.epsilon_decay

In [3]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [7]:
base_model = joblib.load('../base.pkl') 

In [21]:
data = pd.read_csv('train.csv')
data = data.drop(['predicted_weight_g', 'Case', 'obs_time'], axis=1)

In [31]:
test_data = data[:1]
tlqkf = test_data-0.05
base_model.predict(tlqkf)

array([1.78119233])